In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import countDistinct, count
from pyspark.sql.functions import round,ceil


In [0]:
service_credential = dbutils.secrets.get(scope="key-vault-scope1",key="secretkey")


In [0]:



configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "8a00d73d-c9d6-4a27-823f-8fe22afc957b" ,
"fs.azure.account.oauth2.client.secret": service_credential,
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d1c47a41-82ac-441d-91bb-8035c959c41b/oauth2/token"}


In [0]:
dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@olympicdatastorage.dfs.core.windows.net", # container@storage_account_name
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1700628277000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1700628310000


In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/teams.csv")


In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:

#Analysis On Athletes data
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
#Countries with highest number of participating athletes
athletes.groupBy("Country").count().orderBy("count",ascending=False).show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|United States of ...|  615|
|               Japan|  586|
|           Australia|  470|
|People's Republic...|  401|
|             Germany|  400|
|              France|  377|
|              Canada|  368|
|       Great Britain|  366|
|               Italy|  356|
|               Spain|  324|
|                 ROC|  318|
|              Brazil|  291|
|         Netherlands|  274|
|   Republic of Korea|  223|
|         New Zealand|  202|
|              Poland|  195|
|           Argentina|  180|
|        South Africa|  171|
|              Mexico|  155|
|             Hungary|  155|
+--------------------+-----+
only showing top 20 rows



In [0]:

#Most Participated Sports
athletes.groupBy("Discipline").count().orderBy("count",ascending=False).show()

+-----------------+-----+
|       Discipline|count|
+-----------------+-----+
|        Athletics| 2068|
|         Swimming|  743|
|         Football|  567|
|           Rowing|  496|
|           Hockey|  406|
|             Judo|  373|
|         Handball|  343|
|         Shooting|  342|
|          Sailing|  336|
|     Rugby Sevens|  283|
|       Basketball|  280|
|        Wrestling|  279|
|       Volleyball|  274|
|           Boxing|  270|
|       Water Polo|  269|
|          Fencing|  249|
|       Equestrian|  237|
|     Canoe Sprint|  236|
|Baseball/Softball|  220|
|    Cycling Track|  208|
+-----------------+-----+
only showing top 20 rows



In [0]:

#Least Participated Sports
athletes.groupBy("Discipline").count().orderBy("count",ascending=True).show()

+--------------------+-----+
|          Discipline|count|
+--------------------+-----+
|Cycling BMX Frees...|   19|
|Trampoline Gymnas...|   31|
|      Sport Climbing|   37|
|             Surfing|   38|
|  Cycling BMX Racing|   43|
|   Marathon Swimming|   49|
|      3x3 Basketball|   62|
|   Modern Pentathlon|   69|
|Cycling Mountain ...|   74|
|       Skateboarding|   77|
|              Karate|   77|
|        Canoe Slalom|   78|
|    Beach Volleyball|   90|
| Rhythmic Gymnastics|   95|
|   Artistic Swimming|   98|
|           Triathlon|  106|
|                Golf|  115|
|             Archery|  122|
|           Taekwondo|  123|
|              Diving|  133|
+--------------------+-----+
only showing top 20 rows



In [0]:

#Athletes who participate in multiple sports or discipline 
multi_sport_athletes = athletes.groupBy("PersonName").agg(countDistinct("Discipline").alias("Sport_Count")).filter("Sport_Count > 1")
multi_sport_athletes.show()

+--------------------+-----------+
|          PersonName|Sport_Count|
+--------------------+-----------+
|         PEREZ Paola|          2|
|         HAVIK Yoeri|          2|
|       GANNA Filippo|          2|
|       WATANABE Yuta|          2|
|       ALVAREZ Jorge|          2|
|van ROUWENDAAL Sh...|          2|
|         PEREZ Maria|          2|
|PALTRINIERI Gregorio|          2|
|         KIM Hyunsoo|          2|
|           WANG Yang|          2|
|   WELLBROCK Florian|          2|
|          HALL James|          2|
|       MARTIN Daniel|          2|
|       KOPECKY Lotte|          2|
|     KURBANOV Ruslan|          2|
|        DYGERT Chloe|          2|
|          SUN Jiajun|          2|
|             LI Qian|          2|
|           ZHANG Xin|          2|
|       KOVACS Zsofia|          2|
+--------------------+-----------+
only showing top 20 rows



In [0]:
filtered_athletes = athletes.filter((athletes["Country"] == "United States of America") & (athletes["Discipline"] == "Swimming"))

filtered_athletes.show()

+----------------+--------------------+----------+
|      PersonName|             Country|Discipline|
+----------------+--------------------+----------+
|  ANDREW Michael|United States of ...|  Swimming|
|      APPLE Zach|United States of ...|  Swimming|
|ARMSTRONG Joseph|United States of ...|  Swimming|
|    BACON Phoebe|United States of ...|  Swimming|
|    BENTZ Gunnar|United States of ...|  Swimming|
|BRINEGAR Michael|United States of ...|  Swimming|
|     BROWN Erika|United States of ...|  Swimming|
|   CURZAN Claire|United States of ...|  Swimming|
|   de LOOF Catie|United States of ...|  Swimming|
|   DOUGLASS Kate|United States of ...|  Swimming|
|        FINK Nic|United States of ...|  Swimming|
|    FINKE Robert|United States of ...|  Swimming|
| FLICKINGER Hali|United States of ...|  Swimming|
|    GRIMES Katie|United States of ...|  Swimming|
|    HAAS Townley|United States of ...|  Swimming|
|    HARTING Zach|United States of ...|  Swimming|
|   HINDS Natalie|United States

In [0]:
#Analysis on teams data 
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
#How many teams were participated from each country 
teams.groupBy("TeamName","Discipline").count().orderBy("count",ascending=False).show()

+-------------+-------------+-----+
|     TeamName|   Discipline|count|
+-------------+-------------+-----+
|          ROC|     Swimming|    7|
|    Australia|     Swimming|    7|
|United States|     Swimming|    7|
|      Germany|     Swimming|    7|
|        Japan|     Swimming|    7|
|        Italy|     Swimming|    6|
|United States|      Fencing|    6|
|Great Britain|     Swimming|    6|
|          ROC|      Fencing|    6|
|        China|     Swimming|    6|
|       Brazil|     Swimming|    6|
|        Italy|      Fencing|    6|
|       Poland|     Swimming|    6|
|      Germany|Cycling Track|    6|
|       Canada|     Swimming|    6|
|        Japan|      Fencing|    5|
|Great Britain|Cycling Track|    5|
|United States|    Athletics|    5|
|       France|     Swimming|    5|
|        Italy|    Athletics|    5|
+-------------+-------------+-----+
only showing top 20 rows



In [0]:
#How many teams were participated from each country 
teams.groupBy("Discipline").count().orderBy("count",ascending=False).show()

+-------------------+-----+
|         Discipline|count|
+-------------------+-----+
|           Swimming|  113|
|          Athletics|   79|
|      Cycling Track|   64|
|            Archery|   53|
|            Fencing|   52|
|       Table Tennis|   48|
|   Beach Volleyball|   48|
|  Artistic Swimming|   32|
|           Football|   28|
|           Handball|   24|
|         Basketball|   24|
|Artistic Gymnastics|   24|
|             Hockey|   24|
|       Rugby Sevens|   24|
|         Volleyball|   24|
|         Water Polo|   22|
|          Triathlon|   18|
|     3x3 Basketball|   16|
|Rhythmic Gymnastics|   14|
|  Baseball/Softball|   12|
+-------------------+-----+



In [0]:

#Top disciplines in which Women participated 
Women= teams.filter(teams["Event"]=="Women").groupBy("Discipline").count().orderBy("count",ascending=False)

Women.show()

+----------------+-----+
|      Discipline|count|
+----------------+-----+
|Beach Volleyball|   24|
|      Volleyball|   12|
|      Basketball|   12|
|          Hockey|   12|
|    Rugby Sevens|   12|
|        Football|   12|
|        Handball|   12|
|      Water Polo|   10|
|  3x3 Basketball|    8|
+----------------+-----+



In [0]:

#Top disciplines in which Men participated 
Men= teams.filter(teams["Event"]=="Men").groupBy("Discipline").count().orderBy("count",ascending=False)

Men.show()

+----------------+-----+
|      Discipline|count|
+----------------+-----+
|Beach Volleyball|   24|
|        Football|   16|
|      Water Polo|   12|
|      Volleyball|   12|
|    Rugby Sevens|   12|
|      Basketball|   12|
|          Hockey|   12|
|        Handball|   12|
|  3x3 Basketball|    8|
+----------------+-----+



In [0]:
#Disciplines or sports in which Women's Team participated 
womens_team= teams.filter(teams["Event"]=="Women's Team").groupBy("Discipline").count().orderBy("count",ascending=False)

womens_team.show()


+-------------------+-----+
|         Discipline|count|
+-------------------+-----+
|       Table Tennis|   16|
|Artistic Gymnastics|   12|
|            Archery|   12|
+-------------------+-----+



In [0]:

#Analysis on Coaches data
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
#Analysis on coaches data 

#Countries having highest number of coaches 

#coaches.filter(col("Event").isNull()).groupBy("Country").count().orderBy("count",ascending=False).show()

coaches.filter(col("Event")!="NULL").groupBy("Country").count().orderBy("count",ascending=False).show()


+--------------------+-----+
|             Country|count|
+--------------------+-----+
|               Japan|   19|
|           Australia|   17|
|United States of ...|   16|
|              Canada|   15|
|               Spain|   13|
|               Italy|   10|
|        South Africa|    8|
|                 ROC|    8|
|           Argentina|    8|
|              France|    8|
|People's Republic...|    8|
|             Germany|    7|
|              Brazil|    7|
|       Great Britain|    7|
|         New Zealand|    7|
|              Mexico|    7|
|               Egypt|    6|
|         Netherlands|    6|
|   Republic of Korea|    6|
|               India|    5|
+--------------------+-----+
only showing top 20 rows



In [0]:
coaches.groupBy("Name").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|                Name|count|
+--------------------+-----+
|    TARRES CAMPA Ana|    2|
|    GUERRERO Rolando|    2|
| RETEGUI Carlos Jose|    2|
|       ELAFANDI Nour|    2|
|GIALLOMBARDO Patr...|    2|
|REQUENA PERICAS J...|    2|
|   CHEPAK Anastasiya|    2|
|            WANG Jie|    2|
|  CHERNETSKA Natalia|    2|
|    MONTICO Loredana|    2|
|       SZAUDER Gabor|    2|
|       FUJIKI Mayuko|    2|
|   FARINELLI Roberta|    2|
|     KLARENBEEK Auke|    1|
|      ROQUES Anthony|    1|
|     SCARIOLO Sergio|    1|
|  KIRKPATRICK Andrew|    1|
|    JACOBSEN Nikolaj|    1|
|       BREIVIK Marit|    1|
|     GUSTAVSSON Tony|    1|
+--------------------+-----+
only showing top 20 rows



In [0]:
#Distribution of coaches across different countries and sports or discipline 

coach_distribution = coaches.groupBy("Country", "Discipline").count().orderBy("count", ascending=False)
coach_distribution.show()

+--------------------+-----------------+-----+
|             Country|       Discipline|count|
+--------------------+-----------------+-----+
|               Spain|       Basketball|   10|
|             Nigeria|       Basketball|    9|
|United States of ...|       Basketball|    9|
|               Spain|Artistic Swimming|    8|
|               Italy|Artistic Swimming|    7|
|United States of ...|Baseball/Softball|    6|
|           Australia|           Hockey|    6|
|               Chile|         Football|    6|
|           Venezuela|       Basketball|    6|
|               Egypt|Artistic Swimming|    6|
|               Japan|         Football|    6|
|         Netherlands|           Hockey|    5|
|               Egypt|         Football|    5|
|                 ROC|       Volleyball|    5|
|           Australia|       Basketball|    5|
|               Japan|       Basketball|    5|
|       C�te d'Ivoire|         Football|    5|
|United States of ...|       Volleyball|    5|
|        Sout

In [0]:
#How many Football Coaches by country


football_coaches1=coaches.filter((col("Discipline")=="Football") & (col("Event")!="NULL"))

football_coaches2=football_coaches1.groupBy("Country").count().orderBy("count",ascending=False)

football_coaches2.show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|               Japan|    2|
|              Brazil|    2|
|         New Zealand|    2|
|           Australia|    2|
|   Republic of Korea|    1|
|           Argentina|    1|
|            Honduras|    1|
|People's Republic...|    1|
|             Germany|    1|
|              Mexico|    1|
|              Sweden|    1|
|        Saudi Arabia|    1|
|       C�te d'Ivoire|    1|
|       Great Britain|    1|
|              Zambia|    1|
|               Spain|    1|
|               Chile|    1|
|              France|    1|
|United States of ...|    1|
|              Canada|    1|
+--------------------+-----+
only showing top 20 rows



In [0]:
#Analysis on Gender Entries data  
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
#Average number of entries by gender for each sport or discipline

average_entries_by_gender = entriesgender.withColumn('Avg_Female', (entriesgender['Female'] / entriesgender['Total'])*100).withColumn('Avg_Male', (entriesgender['Male'] / entriesgender['Total'])*100)

average_entries_by_gender.show()

+--------------------+------+----+-----+------------------+------------------+
|          Discipline|Female|Male|Total|        Avg_Female|          Avg_Male|
+--------------------+------+----+-----+------------------+------------------+
|      3x3 Basketball|    32|  32|   64|              50.0|              50.0|
|             Archery|    64|  64|  128|              50.0|              50.0|
| Artistic Gymnastics|    98|  98|  196|              50.0|              50.0|
|   Artistic Swimming|   105|   0|  105|             100.0|               0.0|
|           Athletics|   969|1072| 2041| 47.47672709456149| 52.52327290543851|
|           Badminton|    86|  87|  173| 49.71098265895954| 50.28901734104046|
|   Baseball/Softball|    90| 144|  234| 38.46153846153847| 61.53846153846154|
|          Basketball|   144| 144|  288|              50.0|              50.0|
|    Beach Volleyball|    48|  48|   96|              50.0|              50.0|
|              Boxing|   102| 187|  289|35.294117647

In [0]:
#Most participated discipline by both gender

both_gender=entriesgender.sort("Total",ascending=False)

both_gender.show()

+-----------------+------+----+-----+
|       Discipline|Female|Male|Total|
+-----------------+------+----+-----+
|        Athletics|   969|1072| 2041|
|         Swimming|   361| 418|  779|
|         Football|   264| 344|  608|
|           Rowing|   257| 265|  522|
|             Judo|   192| 201|  393|
|           Hockey|   192| 192|  384|
|         Shooting|   178| 178|  356|
|          Sailing|   175| 175|  350|
|         Handball|   168| 168|  336|
|     Rugby Sevens|   146| 151|  297|
|        Wrestling|    96| 193|  289|
|           Boxing|   102| 187|  289|
|       Volleyball|   144| 144|  288|
|       Basketball|   144| 144|  288|
|       Water Polo|   122| 146|  268|
|     Canoe Sprint|   123| 126|  249|
|Baseball/Softball|    90| 144|  234|
|          Fencing|   107| 108|  215|
|     Cycling Road|    70| 131|  201|
|       Equestrian|    73| 125|  198|
+-----------------+------+----+-----+
only showing top 20 rows



In [0]:

#Least participated discipline by both gender
both_gender=entriesgender.sort("Total",ascending=True)

both_gender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|Cycling BMX Frees...|    10|   9|   19|
|Trampoline Gymnas...|    16|  16|   32|
|      Sport Climbing|    20|  20|   40|
|             Surfing|    20|  20|   40|
|  Cycling BMX Racing|    24|  24|   48|
|   Marathon Swimming|    25|  25|   50|
|      3x3 Basketball|    32|  32|   64|
|   Modern Pentathlon|    36|  36|   72|
|Cycling Mountain ...|    38|  38|   76|
|       Skateboarding|    40|  40|   80|
|              Karate|    40|  42|   82|
|        Canoe Slalom|    41|  41|   82|
| Rhythmic Gymnastics|    96|   0|   96|
|    Beach Volleyball|    48|  48|   96|
|   Artistic Swimming|   105|   0|  105|
|           Triathlon|    55|  55|  110|
|                Golf|    60|  60|  120|
|             Archery|    64|  64|  128|
|           Taekwondo|    65|  65|  130|
|              Diving|    72|  71|  143|
+--------------------+------+----+-----+
only showing top

In [0]:
#Analysis on Medals data 
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
#Total medals won by each country

from pyspark.sql import functions as F

total_medals_by_country = medals.groupBy("Team_Country").agg(F.sum("Gold").alias("Total_Gold"),F.sum("Silver").alias("Total_Silver"),F.sum("Bronze").alias("Total_Bronze")
).orderBy("Total_Gold", ascending=False)

total_medals_by_country.show()

+--------------------+----------+------------+------------+
|        Team_Country|Total_Gold|Total_Silver|Total_Bronze|
+--------------------+----------+------------+------------+
|United States of ...|        39|          41|          33|
|People's Republic...|        38|          32|          18|
|               Japan|        27|          14|          17|
|       Great Britain|        22|          21|          22|
|                 ROC|        20|          28|          23|
|           Australia|        17|           7|          22|
|               Italy|        10|          10|          20|
|              France|        10|          12|          11|
|             Germany|        10|          11|          16|
|         Netherlands|        10|          12|          14|
|              Canada|         7|           6|          11|
|                Cuba|         7|           3|           5|
|         New Zealand|         7|           6|           7|
|              Brazil|         7|       

In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
#Top countries with the highest number of silver medals

top_silver_medal_countries= medals.orderBy("Silver",ascending=False).select("Team_country","Silver")
top_silver_medal_countries.show()

+--------------------+------+
|        Team_country|Silver|
+--------------------+------+
|United States of ...|    41|
|People's Republic...|    32|
|                 ROC|    28|
|       Great Britain|    21|
|               Japan|    14|
|         Netherlands|    12|
|              France|    12|
|             Germany|    11|
|               Italy|    10|
|               Spain|     8|
|           Australia|     7|
|             Hungary|     7|
|              Brazil|     6|
|              Canada|     6|
|             Ukraine|     6|
|              Sweden|     6|
|         New Zealand|     6|
|             Georgia|     5|
|              Poland|     5|
|               Kenya|     4|
+--------------------+------+
only showing top 20 rows



In [0]:

#Top countries with the highest number of bronze medals
top_bronze_medal_countries= medals.orderBy("Bronze",ascending=False).select("Team_country","Bronze")
top_bronze_medal_countries.show()

+--------------------+------+
|        Team_country|Bronze|
+--------------------+------+
|United States of ...|    33|
|                 ROC|    23|
|       Great Britain|    22|
|           Australia|    22|
|               Italy|    20|
|People's Republic...|    18|
|               Japan|    17|
|             Germany|    16|
|         Netherlands|    14|
|             Ukraine|    12|
|              France|    11|
|              Canada|    11|
|   Republic of Korea|    10|
|              Turkey|     9|
|              Brazil|     8|
|          Kazakhstan|     8|
|             Hungary|     7|
|         New Zealand|     7|
|      Chinese Taipei|     6|
|         Switzerland|     6|
+--------------------+------+
only showing top 20 rows



In [0]:

#Top countries with highest number of total medals 
top_total_medal_countries= medals.sort("Total",ascending=False).select("Team_country","Total")
top_total_medal_countries.show()

+--------------------+-----+
|        Team_country|Total|
+--------------------+-----+
|United States of ...|  113|
|People's Republic...|   88|
|                 ROC|   71|
|       Great Britain|   65|
|               Japan|   58|
|           Australia|   46|
|               Italy|   40|
|             Germany|   37|
|         Netherlands|   36|
|              France|   33|
|              Canada|   24|
|              Brazil|   21|
|         New Zealand|   20|
|             Hungary|   20|
|   Republic of Korea|   20|
|             Ukraine|   19|
|               Spain|   17|
|                Cuba|   15|
|              Poland|   14|
|         Switzerland|   13|
+--------------------+-----+
only showing top 20 rows



In [0]:

athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/transformed-data/athletes")
     
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches")

entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/entriesgender")

medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals")

teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams")
     
